celery -A celery_app worker --loglevel=info
celery -A celery_app beat --loglevel=info


In [1]:
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import nest_asyncio

import pytesseract
from PIL import Image
import pyautogui
import cv2
import time
import numpy as np
import subprocess
import webbrowser

import sys, os

# Go one level up from the notebook folder
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from img_to_text.ocr import extract_text_strong

# ===== CONFIG =====
URL = "https://portfolio-praveen-gupta.vercel.app/"
# URL = "https://practice.expandtesting.com/contact?utm_source=chatgpt.com"
SCROLL_AMOUNT = -50
SCROLL_DELAY = 0.2
SCROLL_LIMIT = 200

# --- Screenshot helper ---
def take_screenshot():
    screenshot_path = "/home/praveen/Desktop/My-Projects/interview_p/imgs/outputs/screen.png"
    time.sleep(0.5)
    subprocess.run(["gnome-screenshot", "-f", screenshot_path])
    return screenshot_path

# --- OCR extraction ---
def extract_text(image_path):
    return extract_text_strong(image_path)  # use your function directly

# --- Find coordinates of text in image ---
def find_text_coordinate_in_image(image_path, search_text):
    img = Image.open(image_path).convert("L")
    img = img.point(lambda x: 0 if x < 130 else 255)

    boxes = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)
    for i in range(len(boxes['text'])):
        if search_text.lower() in boxes['text'][i].lower():
            x, y, w, h = boxes['left'][i], boxes['top'][i] + 25, boxes['width'][i], boxes['height'][i] + 25
            center_x = x + w // 2
            center_y = y + h // 2
            return (center_x, center_y)
    return None

# --- Fill field and press Tab ---
def fill_field(value):
    pyautogui.typewrite(value)
    pyautogui.press("tab")
    time.sleep(0.5)

# Allow nested event loops (needed for Jupyter/IPython)
nest_asyncio.apply()

async def fetch_form():
    session = AsyncHTMLSession()
    webbrowser.open(URL)

    r = await session.get(URL)
    await r.html.arender(timeout=20, sleep=2)

    soup = BeautifulSoup(r.html.html, "html.parser")
    form = soup.find("form")

    if not form:
        print("[ERROR] No form found on page.")
        return []

    fields_bs4 = form.find_all(["input", "textarea"])
    field_names = []
    for tag in fields_bs4:
        if tag.get("type") in ["submit", "hidden", "checkbox", "radio"]:
            continue
        placeholder = tag.get("name") or tag.get("name") or tag.get("id") or tag.text
        if placeholder:
            field_names.append(placeholder.strip())

    print("[INFO] Required fields:", field_names)
    return field_names

async def main():
    field_names = await fetch_form()
    if not field_names:
        return

    # Prepare input data safely
    input_data = {}
    if len(field_names) >= 3:
        input_data = {
            field_names[0]: "John Doe",
            field_names[1]: "john@example.com",
            field_names[2]: "This is a test message"
        }
    else:
        for name in field_names:
            input_data[name] = f"Sample for {name}"

    # Locate first field by OCR
    target_label = field_names[0]
    coord = None
    for attempt in range(SCROLL_LIMIT):
        shot = take_screenshot()
        coord = find_text_coordinate_in_image(shot, target_label)
        if coord:
            print(f"[FOUND] '{target_label}' at {coord} (attempt {attempt+1})")
            pyautogui.click(coord[0], coord[1])
            fill_field(input_data[target_label])
            input_data[target_label] = ""  # mark filled
            break
        print(f"[INFO] '{target_label}' not found, scrolling...")
        pyautogui.scroll(SCROLL_AMOUNT)
        time.sleep(SCROLL_DELAY)

    # Fill remaining fields sequentially via Tab
    for fname, fvalue in input_data.items():
        if fvalue:
            fill_field(fvalue)

    # Press Enter to submit
    pyautogui.press('enter')

# Run the main function
await main()


[INFO] Required fields: ['name', 'email', 'message']
[INFO] 'name' not found, scrolling...
[FOUND] 'name' at (1012, 513) (attempt 2)


In [ ]:
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import nest_asyncio

import pytesseract
from PIL import Image
import pyautogui
import cv2
import time
import numpy as np
import subprocess
import webbrowser

import sys, os

# Go one level up from the notebook folder
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from img_to_text.ocr import extract_text_strong

# ===== CONFIG =====
URL = "https://portfolio-praveen-gupta.vercel.app/"
URL = "https://practice.expandtesting.com/contact?utm_source=chatgpt.com"
SCROLL_AMOUNT = -50
SCROLL_DELAY = 0.2
SCROLL_LIMIT = 200

# --- Screenshot helper ---
def take_screenshot():
    screenshot_path = "/home/praveen/Desktop/My-Projects/interview_p/imgs/outputs/screen.png"
    time.sleep(0.1)
    subprocess.run(["gnome-screenshot", "-f", screenshot_path])
    return screenshot_path

# --- OCR extraction ---
def extract_text(image_path):
    return extract_text_strong(image_path)  # use your function directly

# --- Find coordinates of text in image ---
def find_text_coordinate_in_image(image_path, search_text):
    img = Image.open(image_path).convert("L")
    img = img.point(lambda x: 0 if x < 130 else 255)

    boxes = pytesseract.image_to_data(img, output_type=pytesseract.Output.DICT)
    for i in range(len(boxes['text'])):
        if search_text.lower() in boxes['text'][i].lower():
            x, y, w, h = boxes['left'][i], boxes['top'][i] + 25, boxes['width'][i], boxes['height'][i] + 25
            center_x = x + w // 2
            center_y = y + h // 2
            return (center_x, center_y)
    return None

# --- Fill field and press Tab ---
def fill_field(value):
    pyautogui.typewrite(value)
    pyautogui.press("tab")
    time.sleep(0.5)

# Allow nested event loops (needed for Jupyter/IPython)
nest_asyncio.apply()

async def fetch_form():
    session = AsyncHTMLSession()
    webbrowser.open(URL)

    r = await session.get(URL)
    await r.html.arender(timeout=20, sleep=1)

    soup = BeautifulSoup(r.html.html, "html.parser")
    

    fields_bs4 = soup.find_all(class_='form-label')
    field_names = []
    for tag in fields_bs4:
        if tag.get("type") in ["submit", "hidden", "checkbox", "radio"]:
            continue
        placeholder = tag.text.strip()
        if placeholder:
            field_names.append(placeholder.strip())

    print("[INFO] Required fields:", field_names)
    return field_names

async def main():
    field_names = await fetch_form()
    if not field_names:
        return

    # Prepare input data safely
    input_data = {}
    if len(field_names) >= 3:
        input_data = {
            field_names[0]: "John Doe",
            field_names[1]: "john@example.com",
            field_names[2]: "This is a test message"
        }
    else:
        for name in field_names:
            input_data[name] = f"Sample for {name}"

    # Locate first field by OCR
    target_label = field_names[0]
    coord = None
    for attempt in range(SCROLL_LIMIT):
        shot = take_screenshot()
        coord = find_text_coordinate_in_image(shot, target_label)
        if coord:
            print(f"[FOUND] '{target_label}' at {coord} (attempt {attempt+1})")
            pyautogui.click(coord[0], coord[1])
            fill_field(input_data[target_label])
            input_data[target_label] = ""  # mark filled
            break
        print(f"[INFO] '{target_label}' not found, scrolling...")
        pyautogui.scroll(SCROLL_AMOUNT)
        time.sleep(SCROLL_DELAY)

    # Fill remaining fields sequentially via Tab
    for fname, fvalue in input_data.items():
        if fvalue:
            fill_field(fvalue)

    # Press Enter to submit
    pyautogui.press('enter')

# Run the main function
await main()


[INFO] Required fields: ['Name', 'Email', 'You message']
[FOUND] 'Name' at (379, 615) (attempt 1)


# another try

In [36]:
from pytesseract import Output
import pytesseract
import cv2
import numpy as np
from PIL import Image, ImageDraw

def find_input_box_coordinate(image_path, label_text,
                              max_search_h=300,
                              min_width=40,
                              min_aspect=2.0,
                              debug_save=None):
    """
    Return screen-coordinate (x_center, y_center) of the input box nearest & below label_text.
    If not found, returns a conservative fallback location (below label) or None.
    debug_save: if provided, will save a debug image with overlayed boxes.
    """

    # read color image and grayscale
    color = cv2.imread(image_path)
    if color is None:
        return None
    h_img, w_img = color.shape[:2]
    gray = cv2.cvtColor(color, cv2.COLOR_BGR2GRAY)

    # OCR to find label bounding box (pick highest-confidence match containing label_text)
    boxes = pytesseract.image_to_data(gray, output_type=Output.DICT)
    best_idx = None
    best_conf = -999.0
    for i, txt in enumerate(boxes['text']):
        if not txt or txt.strip() == "":
            continue
        if label_text.lower() in txt.lower():
            try:
                conf = float(boxes['conf'][i])
            except:
                conf = 0.0
            if conf > best_conf:
                best_conf = conf
                best_idx = i

    if best_idx is None:
        # Label not found
        return None

    lx = int(boxes['left'][best_idx])
    ly = int(boxes['top'][best_idx])
    lw = int(boxes['width'][best_idx])
    lh = int(boxes['height'][best_idx])
    label_bottom = ly + lh

    # Define ROI: area below the label where the input box likely sits
    roi_y1 = label_bottom
    roi_y2 = min(h_img, roi_y1 + max_search_h)
    roi_x1 = max(0, lx - 80)                          # allow some left margin
    roi_x2 = min(w_img, lx + lw + 300)                # allow some right margin

    roi = gray[roi_y1:roi_y2, roi_x1:roi_x2]
    if roi.size == 0:
        return None

    # Preprocess ROI for contour detection
    blur = cv2.GaussianBlur(roi, (3,3), 0)
    th = cv2.adaptiveThreshold(blur, 255,
                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY_INV, 15, 8)  # invert makes boxes white
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel, iterations=1)

    # Find contours in ROI
    contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    candidates = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w < min_width or h < 8:
            continue
        aspect = w / float(h) if h>0 else 0
        if aspect < min_aspect:
            continue
        # map to full-image coordinates
        ax = x + roi_x1
        ay = y + roi_y1
        candidates.append((ax, ay, w, h))

    # Keep only candidates that are below the label
    candidates = [c for c in candidates if c[1] > label_bottom - 2]

    # Score candidates: prefer small vertical distance below label and good horizontal overlap
    def score_candidate(c):
        ax, ay, aw, ah = c
        vert_dist = ay - label_bottom if ay >= label_bottom else 9999
        # horizontal overlap measure (negative if no overlap)
        overlap = max(0, min(ax+aw, lx+lw) - max(ax, lx))
        # prefer overlap; penalize if no overlap
        overlap_penalty = 0 if overlap>0 else 200
        return vert_dist + overlap_penalty

    if candidates:
        candidates.sort(key=score_candidate)
        best = candidates[0]
        bx, by, bw, bh = best
        center_x = bx + bw // 2
        center_y = by + bh // 2

        # save debug image optionally
        if debug_save:
            pil = Image.fromarray(cv2.cvtColor(color, cv2.COLOR_BGR2RGB))
            draw = ImageDraw.Draw(pil)
            # label box (green)
            draw.rectangle([(lx, ly), (lx+lw, ly+lh)], outline="green", width=2)
            # ROI (blue)
            draw.rectangle([(roi_x1, roi_y1), (roi_x2, roi_y2)], outline="blue", width=1)
            # best input (red)
            draw.rectangle([(bx, by), (bx+bw, by+bh)], outline="red", width=3)
            pil.save(debug_save)

        return (center_x, center_y)

    # Fallback: no detected candidate — return conservative point slightly below label
    fallback_x = lx + lw // 2
    fallback_y = label_bottom + max(20, int(lh*1.2))
    if debug_save:
        pil = Image.fromarray(cv2.cvtColor(color, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(pil)
        draw.rectangle([(lx, ly), (lx+lw, ly+lh)], outline="green", width=2)
        draw.text((lx, ly - 12), "fallback", fill="yellow")
        pil.save(debug_save)
    return (fallback_x, fallback_y)


In [37]:
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import nest_asyncio

import pytesseract
from PIL import Image, ImageDraw
import pyautogui
import cv2
import time
import numpy as np
import subprocess
import webbrowser

import sys, os

# Go one level up from the notebook folder
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from img_to_text.ocr import extract_text_strong

# ===== CONFIG =====
URL = "https://portfolio-praveen-gupta.vercel.app/"
URL = "https://practice.expandtesting.com/contact?utm_source=chatgpt.com"
SCROLL_AMOUNT = -50
SCROLL_DELAY = 0.2
SCROLL_LIMIT = 200

# --- Screenshot helper ---
def take_screenshot():
    screenshot_path = "/home/praveen/Desktop/My-Projects/interview_p/imgs/outputs/screen.png"
    time.sleep(0.5)
    subprocess.run(["gnome-screenshot", "-f", screenshot_path])
    return screenshot_path

# --- OCR extraction ---
def extract_text(image_path):
    return extract_text_strong(image_path)  # use your function directly

# --- Find coordinates of text in image ---
###############################

# --- Fill field and press Tab ---
def fill_field(value):
    pyautogui.typewrite(value)
    pyautogui.press("tab")
    time.sleep(0.5)

# Allow nested event loops (needed for Jupyter/IPython)
nest_asyncio.apply()

async def fetch_form():
    session = AsyncHTMLSession()
    webbrowser.open(URL)

    r = await session.get(URL)
    await r.html.arender(timeout=20, sleep=1)

    soup = BeautifulSoup(r.html.html, "html.parser")
    

    fields_bs4 = soup.find_all(class_='form-label')
    field_names = []
    for tag in fields_bs4:
        if tag.get("type") in ["submit", "hidden", "checkbox", "radio"]:
            continue
        placeholder = tag.text.strip()
        if placeholder:
            field_names.append(placeholder.strip())

    print("[INFO] Required fields:", field_names)
    return field_names

async def main():
    field_names = await fetch_form()
    if not field_names:
        return

    # Prepare input data safely
    input_data = {}
    if len(field_names) >= 3:
        input_data = {
            field_names[0]: "John Doe",
            field_names[1]: "john@example.com",
            field_names[2]: "This is a test message"
        }
    else:
        for name in field_names:
            input_data[name] = f"Sample for {name}"

    # Locate first field by OCR
    target_label = field_names[0]
    coord = None
    for attempt in range(SCROLL_LIMIT):
        shot = take_screenshot()
        coord = find_text_coordinate_in_image(shot, target_label)
        if coord:
            print(f"[FOUND] '{target_label}' at {coord} (attempt {attempt+1})")
            pyautogui.click(coord[0], coord[1])
            fill_field(input_data[target_label])
            input_data[target_label] = ""  # mark filled
            break
        print(f"[INFO] '{target_label}' not found, scrolling...")
        pyautogui.scroll(SCROLL_AMOUNT)
        time.sleep(SCROLL_DELAY)

    # Fill remaining fields sequentially via Tab
    for fname, fvalue in input_data.items():
        if fvalue:
            fill_field(fvalue)

    # Press Enter to submit
    pyautogui.press('enter')

# Run the main function
await main()


[INFO] Required fields: ['Name', 'Email', 'You message']
[FOUND] 'Name' at (379, 578) (attempt 1)
